# Fraudulent Job Detection
#### By Imani Thompson

## Overview

This project analyses which job opening features Indeed should consider when determining whether or not they have a fraudulent job listed. Descriptive analysis i did on job posting predictions shows that job descriptions and job requirements together have a higher potential of identifying fraudulent activity. This was determine by the final model which had a 97% accuracy score. Natural language processing techniques were used on this binary classification model. Indeed can use these findings to get rid of fraudulent jobs on their website or give vistor direct access to my website created to predict the status. 

## Business Understanding

My Stakeholder is Indeed, a worldwide employment website, that has thousands of job postings. An issue that they are encountering is the amount of fraudulent job postings. I was hired by this company to create a model to determine the amount of fraudulent job postings on their website. My predictions are important from a business prospective because I will be analysing actual data to get my answer and not just taking educated guesses. Even though I will not be modeling with Indeed data, it will still be useful to them because it uses the same concept. Determining whether or not a job opening is fruadulent is considered a binary classification, therefore it will require a binary classification model. Since job postings consist of a good bit of text I will be using natural language processing to model with text. Indeed would like a strong model that increases the amount of true negatives and have little to no false positives and false negative. A false positive in this context would be my model predicting that a job opening is fraudlent and it being incorrect. A true negative in this context would be my model predicting a job opening is not fraudulent and it being correct. A false negative in this context would be my model predicting a job opening is not fraudulent but it is. Both false positive and negative are bad but which is worst just depends on the situation. I am focused on optimising the accuracy score when modeling. 

## Data Understanding

The data I used for my analyse is from kaggle. This dataset was made in 2020 and it consist of 18,000 rows which are individual job postings. This dataset has both textual information and meta-information about the jobs. Overall this dataset is related to the data analysis questions because it holds all the necessary information to determine what job postings are legit. The target varible within this dataset is the fraudlent column. Other variables included in this dataset are job job title, location, department, salary range, benefits, etc. The varibles that I will focus on are the descriptions and requirements. These two variables are objects and include missing values. 

In [1]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import roc_auc_score, plot_roc_curve
from collections import defaultdict
import nltk
from nltk.probability import FreqDist
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
from nltk import pos_tag
import string
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer,\
HashingVectorizer
from nltk.stem import WordNetLemmatizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [2]:
import pickle
from lime.lime_text import LimeTextExplainer
from sklearn.pipeline import make_pipeline

In [3]:
# Use this to download the stopwords
nltk.download("stopwords")

# Use this to download the wordnet
nltk.download('wordnet')

# Use this to download the parts of speech tagging
nltk.download('tagsets')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/imanithompson/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/imanithompson/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package tagsets to
[nltk_data]     /Users/imanithompson/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/imanithompson/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [4]:
# Reading in the csv file
data = pd.read_csv('../../../../Downloads/Data/fake_job_postings.csv')

In [5]:
# Showing the first 5 rows of the dataframe
data.head()

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


In [6]:
# Showing the columns within the dataset
data.columns

Index(['job_id', 'title', 'location', 'department', 'salary_range',
       'company_profile', 'description', 'requirements', 'benefits',
       'telecommuting', 'has_company_logo', 'has_questions', 'employment_type',
       'required_experience', 'required_education', 'industry', 'function',
       'fraudulent'],
      dtype='object')

In [7]:
# Description of numeric data in Dataframe
data.describe()

,job_id,telecommuting,has_company_logo,has_questions,fraudulent
count,17880.000000,17880.000000,17880.000000,17880.000000,17880.000000
mean,8940.500000,0.042897,0.795302,0.491723,0.048434
std,5161.655742,0.202631,0.403492,0.499945,0.214688
min,1.000000,0.000000,0.000000,0.000000,0.000000
25%,4470.750000,0.000000,1.000000,0.000000,0.000000
50%,8940.500000,0.000000,1.000000,0.000000,0.000000
75%,13410.250000,0.000000,1.000000,1.000000,0.000000
max,17880.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
# Description of object data in Dataframe
data.describe(include='O')

,title,location,department,salary_range,company_profile,description,requirements,benefits,employment_type,required_experience,required_education,industry,function
count,17880,17534,6333,2868,14572,17879,15185,10670,14409,10830,9775,12977,11425
unique,11231,3105,1337,874,1709,14801,11968,6205,5,7,13,131,37
top,English Teacher Abroad,"GB, LND, London",Sales,0-0,We help teachers get safe &amp; secure jobs ab...,"Play with kids, get paid for it Love travel? J...",University degree required. TEFL / TESOL / CEL...,See job description,Full-time,Mid-Senior level,Bachelor's Degree,Information Technology and Services,Information Technology
freq,311,718,551,142,726,379,410,726,11620,3809,5145,1734,1749


In [9]:
# Shows information about the Dataframe
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17880 entries, 0 to 17879
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   job_id               17880 non-null  int64 
 1   title                17880 non-null  object
 2   location             17534 non-null  object
 3   department           6333 non-null   object
 4   salary_range         2868 non-null   object
 5   company_profile      14572 non-null  object
 6   description          17879 non-null  object
 7   requirements         15185 non-null  object
 8   benefits             10670 non-null  object
 9   telecommuting        17880 non-null  int64 
 10  has_company_logo     17880 non-null  int64 
 11  has_questions        17880 non-null  int64 
 12  employment_type      14409 non-null  object
 13  required_experience  10830 non-null  object
 14  required_education   9775 non-null   object
 15  industry             12977 non-null  object
 16  func

In [10]:
# Showing the shape of the dataframe
data.shape

(17880, 18)

## Data Preparation

Since my main focus concerning this datatset is the job description and requiremnts, I am going to ignore all other columns. The columns `decription` and `requirements` have missing values so I dropped all of them. I also made a new column called `words` that is a combination of both description and requirements.

In [11]:
data[['description', 'requirements']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17880 entries, 0 to 17879
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   description   17879 non-null  object
 1   requirements  15185 non-null  object
dtypes: object(2)
memory usage: 279.5+ KB


In [12]:
data = data.dropna(subset=['description', 'requirements'])

In [13]:
data['words'] = data['description'] + ' ' + data['requirements']

In [14]:
# Bring in stopwords
sw = stopwords.words('english')
# Regex pattern to locate 3 capitial letters back to back
caps_pattern = r"([A-Z]{3,})"

In [15]:
def get_wordnet_pos(treebank_tag):
    '''
    Translate nltk POS to wordnet tags
    From phase 4 lecture 6
    '''
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [16]:
def doc_preparer(doc, stop_words=sw):
    '''
    Return: a document string with words which have been 
            lemmatized, 
            parsed for stopwords, 
            made lowercase,
            and stripped of punctuation and numbers.
    From phase 4 lecture 6
    '''
    
    regex_token = RegexpTokenizer(r"([a-zA-Z]+(?:’[a-z]+)?)")
    doc = regex_token.tokenize(doc)
    doc = [word.lower() for word in doc]
    doc = [word for word in doc if word not in sw]
    # print(doc)
    doc = pos_tag(doc)
    doc = [(word[0], get_wordnet_pos(word[1])) for word in doc]
    lemmatizer = WordNetLemmatizer() 
    doc = [lemmatizer.lemmatize(word[0], word[1]) for word in doc]
    return ' '.join(doc)

## Modeling

To arrive at an intial approach, I'm going to analysis my data by first conducting a train test split for each model. Once I have split the data, I will use my `doc_preparer` function to tokenize the text. After the text is clean I will run a second train test split on the tokenized text so that we are able to build a model. Since this is a binary classification I will be using multinomial naive bayes to build models. I will also be using natural language processing since dealing with textual information. 

### Baseline Understanding

In [20]:
# Showings the probablities of fraudulent job openings
y_train.value_counts(normalize=True)

0    0.954689
1    0.045311
Name: fraudulent, dtype: float64

In [21]:
# Making predicts using baseline model
preds_baseline = [0] * len(y_train)
preds_test_baseline = [0] * len(y_test)
# Showing us the accuracy score for modelless baseline
print(f' Accuracy score of train data {accuracy_score(y_train, preds_baseline)}')
print('*' *48)
print(f' Accuracy score of test data {accuracy_score(y_test, preds_test_baseline)}')

 Accuracy score of train data 0.9546891464699684
************************************************
 Accuracy score of test data 0.9483803002370292


A modelless baseline is just taking the majority of the two possibility. For my baseline model, the majority is job openings that are not fraudulent. Because it is majority, the easiest way to determine if a job opeing is fraudulent or not is to assume that it isn't. My binary catgories are very imbalanced, so i will be looking at accuracy as my metric for interperting my model. Based on the accuracy score of the train and test data, this baseline is slightly overfit. The score of the modeless based line is 95.5% but scored on unseen data it is 94.8%.

### Models

#### **Description Model**

In [19]:
# Setting our X and y variables
X = data.description
y = data.fraudulent
# Settign up a train test split
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    random_state=42,
                                                    test_size=0.25)

In [ ]:
# Using yhr doc_preparer function
token_docs = [doc_preparer(doc, sw) for doc in X_train]

In [ ]:
# Example text before the doc_preparer
X_train.iloc[0]

In [ ]:
# Example text after the doc_preparer
token_docs[0]

In [ ]:
# Secondary train-test split to build our best model
X_t, X_val, y_t, y_val = train_test_split(token_docs, y_train,
                                          test_size=0.25, random_state=42)

#### Count Vectorizer & Multinomial Naive Bayes

In [ ]:
# Instantiate count vectorizer and fit on data
cv = CountVectorizer()
X_t_vec = cv.fit_transform(X_t)
X_t_vec  = pd.DataFrame.sparse.from_spmatrix(X_t_vec)
X_t_vec.columns = sorted(cv.vocabulary_)
X_t_vec.set_index(y_t.index, inplace=True)

# We then transform the validation set. (Do not refit the vectorizer!)
X_val_vec = cv.transform(X_val)
X_val_vec  = pd.DataFrame.sparse.from_spmatrix(X_val_vec)
X_val_vec.columns = sorted(cv.vocabulary_)
X_val_vec.set_index(y_val.index, inplace=True)

# Now let's fit the the Multinomial Naive Bayes Classifier on our data and predict
mnb = MultinomialNB()
mnb.fit(X_t_vec, y_t)
# Generate model predictions and plot confusion matrix
y_hat = mnb.predict(X_val_vec)
plot_confusion_matrix(mnb, X_val_vec, y_val);

In [ ]:
print(f'accuracy score {accuracy_score(y_val, y_hat)}')
print(f'f1 score {f1_score(y_val, y_hat)}')

In [ ]:
prior_0 = y_t.value_counts()[0]/len(y_t)
prior_1 = y_t.value_counts()[1]/len(y_t)
print(prior_0, prior_1)

In [ ]:
print(mnb.class_log_prior_)

In [ ]:
print(np.log(prior_0))

- This Multinomial naive bayes model using job descriptions has a accuracy score of 96.8%. This model is 1.8% better than the baseline model. There is a high true negative value like i wanted. The false positive and false negative values are not too bad. 

#### **Requirement Model**

In [ ]:
# Setting up our variables
X = data.requirements
y = data.fraudulent
# Setting up for a train test split
X_trainR, X_testR, y_trainR, y_testR = train_test_split(X,
                                                    y,
                                                    random_state=42,
                                                    test_size=0.25)

In [ ]:
# Using my doc_preparer function
token_docsR = [doc_preparer(doc, sw) for doc in X_trainR]

In [ ]:
# Example text before doc_preparer
X_trainR.iloc[0]

In [ ]:
# Example text after doc_preparer
token_docsR[0]

In [ ]:
# Secondary train-test split to build our best model
X_tR, X_valR, y_tR, y_valR = train_test_split(token_docsR, y_trainR,
                                          test_size=0.25, random_state=42)

#### Count Vectorizer & Multinomial Naive Bayes

In [ ]:
# Instantiate the count vectorizer and fitting the data
cv = CountVectorizer()
X_t_vecR = cv.fit_transform(X_tR)
X_t_vecR  = pd.DataFrame.sparse.from_spmatrix(X_t_vecR)
X_t_vecR.columns = sorted(cv.vocabulary_)
X_t_vecR.set_index(y_tR.index, inplace=True)

# We then transform the validation set. (Do not refit the vectorizer!)
X_val_vecR = cv.transform(X_valR)
X_val_vecR  = pd.DataFrame.sparse.from_spmatrix(X_val_vecR)
X_val_vecR.columns = sorted(cv.vocabulary_)
X_val_vecR.set_index(y_valR.index, inplace=True)

# Now let's fit the the Multinomial Naive Bayes Classifier on our data and predict
mnb = MultinomialNB()
mnb.fit(X_t_vecR, y_tR)
# Generate model predictions and plot confusion matrix
y_hatR = mnb.predict(X_val_vecR)
plot_confusion_matrix(mnb, X_val_vecR, y_valR);

In [ ]:
print(f'accuracy score {accuracy_score(y_valR, y_hatR)}')
print(f'precision score {precision_score(y_valR, y_hatR)}')

In [ ]:
prior_0 = y_tR.value_counts()[0]/len(y_tR)
prior_1 = y_tR.value_counts()[1]/len(y_tR)
print(prior_0, prior_1)

In [ ]:
mnb.class_log_prior_

In [ ]:
np.log(prior_0)

- This Multinomial naive bayes model using job requirements has a accuracy score of 96.3%. This model is 1.3% better than the baseline model. The requirements model is slightly worst than the description model. There is a high true negative value like i wanted, higher than the description model too. The false positive and false negative values are not too bad.

#### **Final Model**

#### **Description and Requirements Model**

In [ ]:
# Using the regex caps_pattern
data['words'] = data['words'].str.replace(caps_pattern, r" ").str.strip()

In [ ]:
# Setting the variables
X = data['words']
y = data['fraudulent']
# Setting up for a train test split
X_trainDR, X_testDR, y_trainDR, y_testDR = train_test_split(X,
                                                    y,
                                                    random_state=42,
                                                    test_size=0.25)

In [ ]:
# Using the doc_preparer function
token_docsDR = [doc_preparer(doc, sw) for doc in X_trainDR]

In [ ]:
# Example text before the doc_preparer
X_trainDR[0]

In [ ]:
# Example text after the doc_preparer
token_docsDR[1]

In [ ]:
# Secondary train-test split to build our best model
X_tDR, X_valDR, y_tDR, y_valDR = train_test_split(token_docsDR, y_train,
                                          test_size=0.25, random_state=42)

#### Count Vectorizer & Multinomial Naive Bayes

In [ ]:
# Instantiate the count vectorizer and fit on data
cv = CountVectorizer()
X_t_vecDR = cv.fit_transform(X_tDR)
X_t_vecDR = pd.DataFrame.sparse.from_spmatrix(X_t_vecDR)
X_t_vecDR.columns = sorted(cv.vocabulary_)
X_t_vecDR.set_index(y_tDR.index, inplace=True)

# We then transform the validation set. (Do not refit the vectorizer!)
X_val_vecDR = cv.transform(X_valDR)
X_val_vecDR  = pd.DataFrame.sparse.from_spmatrix(X_val_vecDR)
X_val_vecDR.columns = sorted(cv.vocabulary_)
X_val_vecDR.set_index(y_valDR.index, inplace=True)

# Now let's fit the the Multinomial Naive Bayes Classifier on our data and predict
mnb1 = MultinomialNB()
mnb1.fit(X_t_vecDR, y_tDR)
# Generate model predictions and plot confusion matrix
y_hatDR = mnb1.predict(X_val_vecDR)
plot_confusion_matrix(mnb1, X_val_vecDR, y_valDR);

In [ ]:
print(f'accuracy score {accuracy_score(y_valDR, y_hatDR)}')
print(f'precision score {precision_score(y_valDR, y_hatDR)}')

In [ ]:
prior_0 = y_tDR.value_counts()[0]/len(y_tDR)
prior_1 = y_tDR.value_counts()[1]/len(y_tDR)
print(prior_0, prior_1)

In [ ]:
mnb.class_log_prior_

In [ ]:
np.log(prior_0)

- This Multinomial naive bayes model using both job description and requirements has a accuracy score of 97.2%. This model is 2% better than the baseline model. There is a high true negative value like i wanted, higher than both other models. The false positive and false negative values are not too bad.

### Saving Final Model & Count Vectorizer

In [ ]:
# Saving final MNB model
pickle.dump(mnb1, open("cv_model.sav", 'wb'))

In [ ]:
# Saving count vectorizer
pickle.dump(cv, open("vectorizer.sav", 'wb'))

In [ ]:
# Setting a variable to the final model 
loaded_model = pickle.load(open("cv_model.sav", 'rb'))

In [ ]:
# Making sure it is predicting
loaded_model.predict(X_val_vecDR)

In [ ]:
# Creating a pipeline for count vectorizer and model
pipe = make_pipeline(cv, mnb1)

In [ ]:
pipe.predict_proba([token_docsDR[3]])

In [ ]:
# Instantiating the Lime text explainer
explainer = LimeTextExplainer(class_names=['Not Fraudulent','Fraudulent'])
exp = explainer.explain_instance(token_docsDR[3], 
                                 pipe.predict_proba, 
                                 num_features=20)

print('Probability Fraudulent=', pipe.predict_proba([token_docsDR[3]])[0,1])

In [ ]:
exp.show_in_notebook(text=True)

In [ ]:
exp.as_pyplot_figure()

## Conclusion

I would recommend that Indeed focus on looking more into both the description and requirements together for a higher chance of determining whether it is fraudulent or not. I would also recommend them to give their viewers direct access to my website that I created to make predictions based on both description and requirements. If a viewer is feeling suspicous about a job all they have to do it click a link that takes them directly to my website and paste in their text. A reason why my model has a possibility of not solving the problem everytime is because it is only 97% accurate which leaves 3% of the time being inaccurate. Future plans that I could improve is cleaning the text more, this process is never ending and adjustments can always be made. I would also like to increase true positives, which in this case is if the model predict an opening is fraulent and it is correct. 